<a href="https://colab.research.google.com/github/lisabroadhead/data_science_machine-learning/blob/main/first_model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Model (Practice)
- Lisa Broadhead
- June 27, 2022

* Didn't have a data set to use so I used Raw_Medical_Data_for_day1.csv (we got it from another code along or assigment)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn import set_config
set_config(display='diagram')

In [32]:
file = '/content/drive/MyDrive/Colab Notebooks/coding_dojo/Machine Learning/files/insurance - insurance.csv'

df = pd.read_csv(file)
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [38]:
X = df.drop(columns='charges')
y = df['charges']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### 1. Create a preprocessing object, such as a column transformer or pipeline, that will
- Ordinal encode any ordinal features
- One-hot encode any nominal features 
- Scale any numeric features

In [52]:
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

In [53]:
ohe_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
scaler = StandardScaler()

In [54]:
# Tuples for Column Transformer
cat_tuple = (ohe_encoder, cat_selector)
num_tuple = (scaler, num_selector)

In [55]:
# ColumnTransformer
preprocessor = make_column_transformer(num_tuple, cat_tuple)

### Instantiate a linear regression model

In [59]:
lin_reg = LinearRegression()

lin_reg_pipe = make_pipeline(preprocessor, lin_reg)

lin_reg_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f397342da50>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f397342d350>)])),
                ('linearregression', LinearRegression())])

In [64]:
train_pred = lin_reg_pipe.predict(X_train)
test_pred = lin_reg_pipe.predict(X_test)

In [66]:
train_rsme = r2_score(y_train, train_pred)
test_rsme = r2_score(y_test, test_pred)


print(train_rsme)
print(test_rsme)

0.7449555328228536
0.7672642952734356
